In [4]:
%load_ext autoreload
%autoreload 2


from torch.utils.data import DataLoader, Dataset

import sys
sys.path.append('../')

from sparse_speech_experiment import ContinuousSpeechExperiment

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
data_dir = "/Users/afisher/nta/nupic.research/projects/whydense/gsc/notebooks/data_classes/"

In [51]:
config["batch_size"]

16

In [44]:
from nupic.research.support import parse_config
config_file = "../experiments.cfg"
exp = "sparseCNN2"
config_init = parse_config(config_file)
config = config_init[exp]
config["name"] = exp
config["seed"] = 42
# config["data_dir"] = "/Users/afisher/nta/nupic.torch/examples/gsc/data/"
config["data_dir"] = "/Users/afisher/nta/nupic.research/projects/continuous_learning/notebooks/data_classes/"

In [48]:
experiment = ClasswiseSpeechExperiment(config=config)

Creating optimizer with learning rate= 0.01


In [ ]:
for epoch in range(2):
    experiment.train(epoch, 1)

In [59]:
experiment.test(7)

{'total_correct': 0,
 'mean_loss': 8.309527065815717,
 'mean_accuracy': 0.0,
 'entropy': 26588.201171875,
 'total_samples': 7564,
 'non_zero_parameters': 194452}

In [ ]:
"/Users/afisher/nta/nupic.torch/"

In [ ]:
class ClasswiseDataset(PreprocessedDataset):
    def load_qualifier(self, qualifier):
        """
        Call this to load the a copy of a dataset with the specific qualifier into
        memory.

        :return: Name of the file that was actually loaded.
        """
        file_name = os.path.join(self.path, self.basename + "{}.npz".format(qualifier))
#         self.tensors = list(np.load(file_name, allow_pickle=True))
        self.tensors = list(torch.load(file_name))
        return file_name
    
class ClasswiseGSCExperiment(SparseSpeechExperiment):
    def __init__(self, class_):
        self.class_ = class_
        
    def load_datasets(self, class_):
        validation_dataset = ClasswiseDataset(
            cachefilepath=self.data_dir,
            basename="data_valid",
            qualifiers=[''],
        )

        test_dataset = ClasswiseDataset(
            cachefilepath=self.data_dir,
            basename="data_test_",
            qualifiers=range(class_,class_+1),
        )
        train_dataset = ClasswiseDataset(
            cachefilepath=self.data_dir,
            basename="data_train_",
            qualifiers=range(class_,class_+1),
        )
        
        
        self.train_loader = DataLoader(
            train_dataset, batch_size=self.batch_size, shuffle=True
        )

        self.validation_loader = DataLoader(
            validation_dataset, batch_size=self.batch_size, shuffle=False
        )

        self.test_loader = DataLoader(
            test_dataset, batch_size=self.batch_size, shuffle=False
        )